In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import os


imagenet_stats = {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}

def normalize_normal(normals):
    normals_norm = torch.sqrt(torch.sum(torch.pow(normals, 2), 1)).unsqueeze(1).repeat(1, 3, 1, 1) + 1e-8
    return torch.div(normals, normals_norm)

def normalize_coords(coords):
    coord_n = coords[:, 0:3, :, :]
    coord_u = coords[:, 3:6, :, :]
    return torch.cat((normalize_normal(coord_n), normalize_normal(coord_u)), 1)

class conv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, stride):
        super(conv, self).__init__()
        self.kernel_size = kernel_size
        self.conv_base = nn.Conv2d(num_in_layers, num_out_layers, kernel_size=kernel_size, stride=stride)
        self.normalize = nn.BatchNorm2d(num_out_layers)

    def forward(self, x):
        p = int(np.floor((self.kernel_size - 1) / 2))
        p2d = (p, p, p, p)
        x = self.conv_base(F.pad(x, p2d))
        x = self.normalize(x)
        return F.elu(x, inplace=True)
    

class upconv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, scale):
        super(upconv, self).__init__()
        self.scale = scale
        self.conv1 = conv(num_in_layers, num_out_layers, kernel_size, 1)

    def forward(self, x):
        x = nn.functional.interpolate(x, scale_factor=self.scale, mode='bilinear', align_corners=True)
        return self.conv1(x)
class ResnetModel3HeadsSplitNormalizationTightCoupled(nn.Module):
    def __init__(self, num_in_layers, encoder, pretrained):
        super(ResnetModel3HeadsSplitNormalizationTightCoupled, self).__init__()
        self.pretrained = pretrained
        assert encoder in ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'], "Incorrect encoder type"
        if encoder in ['resnet18', 'resnet34']:
            filters = [64, 128, 256, 512]
        else:
            filters = [256, 512, 1024, 2048]
        resnet = class_for_name("torchvision.models", encoder)(pretrained=pretrained)
        if num_in_layers != 3:  # Number of input channels
            self.firstconv = nn.Conv2d(num_in_layers, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        else:
            self.firstconv = resnet.conv1  # H/2
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool  # H/4

        # encoder
        self.encoder1 = resnet.layer1  # H/4
        self.encoder2 = resnet.layer2  # H/8
        self.encoder3 = resnet.layer3  # H/16
        self.encoder4 = resnet.layer4  # H/32

        # decoder camera
        self.upconv6_cam = upconv(filters[3], 512, 3, 2)
        self.iconv6_cam = conv(filters[2] + 512, 512, 3, 1)
        self.upconv5_cam = upconv(512, 256, 3, 2)
        self.iconv5_cam = conv(filters[1] + 256, 256, 3, 1)
        self.upconv4_cam = upconv(256, 128, 3, 2)
        self.iconv4_cam = conv(filters[0] + 128, 128, 3, 1)
        self.upconv3_cam = upconv(128, 64, 3, 1)
        self.iconv3_cam = conv(64 + 64, 64, 3, 1)
        self.upconv2_cam = upconv(64, 32, 3, 1)
        self.iconv2_cam = conv(64 + 32, 32, 3, 1)
        self.upconv1_cam = upconv(32, 16, 3, 1)
        self.iconv1_cam = conv(16, 16, 3, 1)
        self.nu_layer_cam = get_normal(16, 9)

    def forward(self, x):
        # normalize to imagenet statistics
        x = self.normalize_input(x)

        # encoder
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x_pool1 = self.firstmaxpool(x)

        x_pool2 = self.encoder1(x_pool1)
        x_pool3 = self.encoder2(x_pool2)
        x_pool4 = self.encoder3(x_pool3)
        x_pool5 = self.encoder4(x_pool4)

        # camera
        x_up6_cam = self.upconv6_cam(x_pool5)
        print("x_pool4 size:", x_pool4.size())
        print("x_up6_cam size:", x_up6_cam.size())
        x_up6_cam = torch.cat((x_pool4, x_up6_cam), 1)
        x_iconv6_cam = self.iconv6_cam(x_up6_cam)

        x_up5_cam = self.upconv5_cam(x_iconv6_cam)
        print("x_pool3 size:", x_pool3.size())
        print("x_up5_cam size:", x_up5_cam.size())
        x_up5_cam = torch.cat((x_pool3, x_up5_cam), 1)
        x_iconv5_cam = self.iconv5_cam(x_up5_cam)

        x_up4_cam = self.upconv4_cam(x_iconv5_cam)
        print("x_pool2 size:", x_pool2.size())
        print("x_up4_cam size:", x_up4_cam.size())
        x_up4_cam = torch.cat((x_pool2, x_up4_cam), 1)
        x_iconv4_cam = self.iconv4_cam(x_up4_cam)

        x_up3_cam = self.upconv3_cam(x_iconv4_cam)
        print("x_pool1 size:", x_pool1.size())
        print("x_up3_cam size:", x_up3_cam.size())
        x_up3_cam = torch.cat((x_pool1, x_up3_cam), 1)
        x_iconv3_cam = self.iconv3_cam(x_up3_cam)

        x_up2_cam = self.upconv2_cam(x_iconv3_cam)
        x_up2_cam = torch.cat((x_pool1, x_up2_cam), 1)
        x_iconv2_cam = self.iconv2_cam(x_up2_cam)

        x_up1_cam = self.upconv1_cam(x_iconv2_cam)
        x_iconv1_cam = self.iconv1_cam(x_up1_cam)
        nu_cam = self.nu_layer_cam(x_iconv1_cam)

        # return camera geometry image
        return normalize_coords(nu_cam)

    def normalize_input(self, x):
        # normalize to imagenet statistics
        mean = torch.tensor(imagenet_stats['mean']).view(1, 3, 1, 1).to(x.device)
        std = torch.tensor(imagenet_stats['std']).view(1, 3, 1, 1).to(x.device)
        return (x - mean) / std

def get_normal(in_channels, out_channels):
    return nn.Sequential(
        conv(in_channels, in_channels, 3, 1),
        nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, stride=1, bias=True)
    )

def class_for_name(module_name, class_name):
    # load the module, will raise ImportError if module cannot be loaded
    m = importlib.import_module(module_name)
    # get the class, will raise AttributeError if class cannot be found
    c = getattr(m, class_name)
    return c

def create_model(num_in_layers, encoder='resnet50', pretrained=None):
    # modify the model here
    if encoder == 'resnet50':
        model = ResnetModel3HeadsSplitNormalizationTightCoupled(num_in_layers, encoder, pretrained)
    else:
        raise ValueError("This model has not been implemented yet.")
    return model
def main(input_image_path, output_lsg_folder):
    num_in_layers = 3  # Number of input channels (e.g., 3 for RGB)
    encoder = 'resnet50'  # Encoder type
    pretrained = True  # Use pretrained weights
    model = create_model(num_in_layers, encoder, pretrained)
    print(model)

    # Load the input image
    original_image = cv2.imread(input_image_path)
    if original_image is None:
        print(f"Error: Unable to load image '{input_image_path}'")
        return

    # Forward pass to generate LSG
    input_image = cv2.resize(original_image, (256, 256))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = transforms.ToTensor()(input_image).unsqueeze(0)
    with torch.no_grad():
        camera_normals = model(input_image)

    # Convert camera normals to numpy arrays
    normals_xyz = camera_normals[:, : 5]
    normals_xyz = normals_xyz.squeeze(0).permute(1, 2, 0)  # Reshape to (H, W, C)
    normals_xyz_np = normals_xyz.cpu().numpy()

    # Calculate the magnitude of the vectors
    magnitude = np.sqrt(np.sum(normals_xyz_np ** 2, axis=2))

    # Normalize the magnitude to range [0, 1]
    magnitude_normalized = (magnitude - magnitude.min()) / (magnitude.max() - magnitude.min())

    # Resize the generated image to match the size of the original image
    generated_image_resized = cv2.resize(np.uint8(255*magnitude_normalized), (original_image.shape[1], original_image.shape[0]))

    # Get the filename of the input image without the extension
    input_image_filename = os.path.splitext(os.path.basename(input_image_path))[0]

    # Create the output path for the LSG image
    output_lsg_path = os.path.join(output_lsg_folder, f"{input_image_filename}_LSG.jpg")

    # Save the generated LSG image
    cv2.imwrite(output_lsg_path, generated_image_resized)
    print(f"LSG image saved at: {output_lsg_path}")

if __name__ == '__main__':
    input_image_path = "/home/tr_swapna/Dhyanesh/Demo/1f4c3alg_sachin-tendulkar-_625x300_15_January_24.jpeg"
    output_lsg_path = "/home/tr_swapna/Dhyanesh/Demo_lsg/"
    main(input_image_path, output_lsg_path)



/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to pas

ResnetModel3HeadsSplitNormalizationTightCoupled(
  (firstconv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (firstbn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (firstrelu): ReLU(inplace=True)
  (firstmaxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (encoder1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import os


imagenet_stats = {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}

def normalize_normal(normals):
    normals_norm = torch.sqrt(torch.sum(torch.pow(normals, 2), 1)).unsqueeze(1).repeat(1, 3, 1, 1) + 1e-8
    return torch.div(normals, normals_norm)

def normalize_coords(coords):
    coord_n = coords[:, 0:3, :, :]
    coord_u = coords[:, 3:6, :, :]
    return torch.cat((normalize_normal(coord_n), normalize_normal(coord_u)), 1)

class conv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, stride):
        super(conv, self).__init__()
        self.kernel_size = kernel_size
        self.conv_base = nn.Conv2d(num_in_layers, num_out_layers, kernel_size=kernel_size, stride=stride)
        self.normalize = nn.BatchNorm2d(num_out_layers)

    def forward(self, x):
        p = int(np.floor((self.kernel_size - 1) / 2))
        p2d = (p, p, p, p)
        x = self.conv_base(F.pad(x, p2d))
        x = self.normalize(x)
        return F.elu(x, inplace=True)
    

class upconv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, scale):
        super(upconv, self).__init__()
        self.scale = scale
        self.conv1 = conv(num_in_layers, num_out_layers, kernel_size, 1)

    def forward(self, x):
        x = nn.functional.interpolate(x, scale_factor=self.scale, mode='bilinear', align_corners=True)
        return self.conv1(x)
class ResnetModel3HeadsSplitNormalizationTightCoupled(nn.Module):
    def __init__(self, num_in_layers, encoder, pretrained):
        super(ResnetModel3HeadsSplitNormalizationTightCoupled, self).__init__()
        self.pretrained = pretrained
        assert encoder in ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'], "Incorrect encoder type"
        if encoder in ['resnet18', 'resnet34']:
            filters = [64, 128, 256, 512]
        else:
            filters = [256, 512, 1024, 2048]
        resnet = class_for_name("torchvision.models", encoder)(pretrained=pretrained)
        if num_in_layers != 3:  # Number of input channels
            self.firstconv = nn.Conv2d(num_in_layers, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        else:
            self.firstconv = resnet.conv1  # H/2
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool  # H/4

        # encoder
        self.encoder1 = resnet.layer1  # H/4
        self.encoder2 = resnet.layer2  # H/8
        self.encoder3 = resnet.layer3  # H/16
        self.encoder4 = resnet.layer4  # H/32

        # decoder camera
        self.upconv6_cam = upconv(filters[3], 512, 3, 2)
        self.iconv6_cam = conv(filters[2] + 512, 512, 3, 1)
        self.upconv5_cam = upconv(512, 256, 3, 2)
        self.iconv5_cam = conv(filters[1] + 256, 256, 3, 1)
        self.upconv4_cam = upconv(256, 128, 3, 2)
        self.iconv4_cam = conv(filters[0] + 128, 128, 3, 1)
        self.upconv3_cam = upconv(128, 64, 3, 1)
        self.iconv3_cam = conv(64 + 64, 64, 3, 1)
        self.upconv2_cam = upconv(64, 32, 3, 1)
        self.iconv2_cam = conv(64 + 32, 32, 3, 1)
        self.upconv1_cam = upconv(32, 16, 3, 1)
        self.iconv1_cam = conv(16, 16, 3, 1)
        self.nu_layer_cam = get_normal(16, 9)

    def forward(self, x):
        # normalize to imagenet statistics
        x = self.normalize_input(x)

        # encoder
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x_pool1 = self.firstmaxpool(x)

        x_pool2 = self.encoder1(x_pool1)
        x_pool3 = self.encoder2(x_pool2)
        x_pool4 = self.encoder3(x_pool3)
        x_pool5 = self.encoder4(x_pool4)

        # camera
        x_up6_cam = self.upconv6_cam(x_pool5)
        print("x_pool4 size:", x_pool4.size())
        print("x_up6_cam size:", x_up6_cam.size())
        x_up6_cam = torch.cat((x_pool4, x_up6_cam), 1)
        x_iconv6_cam = self.iconv6_cam(x_up6_cam)

        x_up5_cam = self.upconv5_cam(x_iconv6_cam)
        print("x_pool3 size:", x_pool3.size())
        print("x_up5_cam size:", x_up5_cam.size())
        x_up5_cam = torch.cat((x_pool3, x_up5_cam), 1)
        x_iconv5_cam = self.iconv5_cam(x_up5_cam)

        x_up4_cam = self.upconv4_cam(x_iconv5_cam)
        print("x_pool2 size:", x_pool2.size())
        print("x_up4_cam size:", x_up4_cam.size())
        x_up4_cam = torch.cat((x_pool2, x_up4_cam), 1)
        x_iconv4_cam = self.iconv4_cam(x_up4_cam)

        x_up3_cam = self.upconv3_cam(x_iconv4_cam)
        print("x_pool1 size:", x_pool1.size())
        print("x_up3_cam size:", x_up3_cam.size())
        x_up3_cam = torch.cat((x_pool1, x_up3_cam), 1)
        x_iconv3_cam = self.iconv3_cam(x_up3_cam)

        x_up2_cam = self.upconv2_cam(x_iconv3_cam)
        x_up2_cam = torch.cat((x_pool1, x_up2_cam), 1)
        x_iconv2_cam = self.iconv2_cam(x_up2_cam)

        x_up1_cam = self.upconv1_cam(x_iconv2_cam)
        x_iconv1_cam = self.iconv1_cam(x_up1_cam)
        nu_cam = self.nu_layer_cam(x_iconv1_cam)

        # return camera geometry image
        return normalize_coords(nu_cam)

    def normalize_input(self, x):
        # normalize to imagenet statistics
        mean = torch.tensor(imagenet_stats['mean']).view(1, 3, 1, 1).to(x.device)
        std = torch.tensor(imagenet_stats['std']).view(1, 3, 1, 1).to(x.device)
        return (x - mean) / std

def get_normal(in_channels, out_channels):
    return nn.Sequential(
        conv(in_channels, in_channels, 3, 1),
        nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, stride=1, bias=True)
    )

def class_for_name(module_name, class_name):
    # load the module, will raise ImportError if module cannot be loaded
    m = importlib.import_module(module_name)
    # get the class, will raise AttributeError if class cannot be found
    c = getattr(m, class_name)
    return c

def create_model(num_in_layers, encoder='resnet50', pretrained=None):
    # modify the model here
    if encoder == 'resnet50':
        model = ResnetModel3HeadsSplitNormalizationTightCoupled(num_in_layers, encoder, pretrained)
    else:
        raise ValueError("This model has not been implemented yet.")
    return model

def main():
    num_in_layers = 3  # Number of input channels (e.g., 3 for RGB)
    encoder = 'resnet50'  # Encoder type
    pretrained = True  # Use pretrained weights
    model = create_model(num_in_layers, encoder, pretrained)
    print(model)

    # Path to the input image
    input_image_path = r"/home/tr_swapna/Dhyanesh/Demo/1f4c3alg_sachin-tendulkar-_625x300_15_January_24.jpeg"

    # Output folder for the watermarked image
    output_folder = r"/home/tr_swapna/Dhyanesh/Demo_lsg"

    # Load the input image
    original_image = cv2.imread(input_image_path)
    if original_image is None:
        print(f"Error: Unable to load input image '{input_image_path}'")
        return

    # Forward pass to get the Local Surface Geometry (LSG) for the input image
    input_image = cv2.resize(original_image, (256, 256))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image_tensor = transforms.ToTensor()(input_image).unsqueeze(0)
    with torch.no_grad():
        camera_normals = model(input_image_tensor)

    # Convert camera normals to numpy arrays
    normals_xyz = camera_normals[:, : 3]
    normals_xyz = normals_xyz.squeeze(0).permute(1, 2, 0)  # Reshape to (H, W, C)
    normals_xyz_np = normals_xyz.cpu().numpy()

    # Calculate the magnitude of the vectors
    magnitude = np.sqrt(np.sum(normals_xyz_np ** 2, axis=2))

    # Normalize the magnitude to range [0, 1]
    magnitude_normalized = (magnitude - magnitude.min()) / (magnitude.max() - magnitude.min())

    # Resize the generated image to match the size of the original image
    generated_image_resized = cv2.resize(np.uint8(255*magnitude_normalized), (original_image.shape[1], original_image.shape[0]))

    # Create a green tint mask for the original image
    green_tint_mask = np.zeros_like(original_image, dtype=np.uint8)
    green_tint_mask[:, :, 1] = 255  # Set the green channel to 255 (full green)

    # Overlay green tint mask onto the original image
    alpha = 0.2  # Adjust the transparency of the green tint
    watermarked_image = cv2.addWeighted(original_image, 1 - alpha, green_tint_mask, alpha, 0)

    # Resize the watermarked image to match the dimensions of the generated image
    watermarked_image_resized = cv2.resize(watermarked_image, (generated_image_resized.shape[1], generated_image_resized.shape[0]))

    # Ensure both images have the same dimensions and channels
    watermarked_image_resized = cv2.cvtColor(watermarked_image_resized, cv2.COLOR_RGB2BGR)
    generated_image_resized = cv2.cvtColor(generated_image_resized, cv2.COLOR_RGB2BGR)

    # Combine the watermarked image with the generated LSG image
    combined_image = cv2.addWeighted(watermarked_image_resized, 0.5, generated_image_resized, 0.5, 0)

    # Save the combined image to the output folder
    image_name = os.path.basename(input_image_path)
    combined_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(combined_image_path, combined_image)
    print(f"Combined image saved to: {combined_image_path}")

if __name__ == '__main__':
    main()


/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to pas

ResnetModel3HeadsSplitNormalizationTightCoupled(
  (firstconv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (firstbn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (firstrelu): ReLU(inplace=True)
  (firstmaxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (encoder1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(

In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import os

# Define a function to load the image and perform transformations

def create_resnet_model(num_classes):
    model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def load_and_transform_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),  # Convert images to RGB if they are grayscale
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Define a function to load the ResNet model with the saved weights
def load_resnet_model(weights_path):
    model = create_resnet_model(num_classes=2)  # Create the same model architecture
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model

# Define a function to predict whether the image is fake or real
def predict_image_status(image_path, model):
    image = load_and_transform_image(image_path)
    with torch.no_grad():
        outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    class_idx = predicted.item()
    if class_idx == 0:
        return "Fake"
    elif class_idx == 1:
        return "Real"
    else:
        return "Unknown"

def main():
    # Path to the saved model weights
    weights_path = r"/home/tr_swapna/Dhyanesh/resnet_model.pth"

    # Load the ResNet model with the saved weights
    resnet_model = load_resnet_model(weights_path)

    # Path to the single image you want to predict
    image_path_to_predict = r"/home/tr_swapna/Dhyanesh/Demo_lsg/1f4c3alg_sachin-tendulkar-_625x300_15_January_24.jpeg"

    # Predict the status (fake/real) of the image
    predicted_status = predict_image_status(image_path_to_predict, resnet_model)
    print(f"The predicted status of the image '{image_path_to_predict}' is: {predicted_status}")

if __name__ == "__main__":
    main()


Downloading: "https://github.com/pytorch/vision/zipball/v0.9.0" to /home/tr_swapna/.cache/torch/hub/v0.9.0.zip
/home/tr_swapna/anaconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The predicted status of the image '/home/tr_swapna/Dhyanesh/Demo_lsg/1f4c3alg_sachin-tendulkar-_625x300_15_January_24.jpeg' is: Fake
